In [1]:
using LinearAlgebra
using Random
using PlotlyJS

include("kalmanflt.jl");

WebIO._IJuliaInit()

The system is a particle falling due to gravity WITH air resistance. For this simulation, the falling object will acheive terminal velocity. The velocity as a function of time is modeled as

$$
v(t) = v_{\infty} \tanh\left(\frac{g t}{v_{\infty}}\right)
$$

where $v_{\infty}$ is the terminal velocity as time approaches infinity and $g$ is the acceleration due to gravity. We want to model the position and velocity of an object falling, but we cannot use the following equations

$$
\begin{align}
h &= h_{0} + v_0 t + \frac{1}{2} a t^{2} \\
v &= v_{0} + g t
\end{align}
$$

since the acceleration is NOT constant

$$
\begin{align}
a &= \frac{dv}{dt} = g \ \text{sech}^2\left(\frac{g t}{v_{\infty}}\right)
\end{align}
$$

Let's consider the following model instead

$$
\begin{align}
h \rightarrow h + \delta h &= \sum\limits_{n=0}^{\infty} \frac{1}{n!}\left.\left(\frac{d^{n}h}{dt^n}\right)\right|_{t=T}\left(\Delta t\right)^n \\
     &= h(T) + v(T) \Delta t + \frac{1}{2} a(T) \left(\Delta t\right)^2 + \frac{1}{3!} j(T) \left(\Delta t\right)^3 + \frac{1}{4!} s(T) \left(\Delta t\right)^4 + \mathcal{O}(t^5)
\end{align}
$$

$$
\begin{align}
v(t) &= v_{\infty} \tanh\left(\frac{g t}{v_{\infty}}\right) \\
a(t) &= \frac{dv}{dt}     = g \ \text{sech}^2\left(\frac{g t}{v_{\infty}}\right) \\
j(t) &= \frac{d^2v}{dt^2} = -\frac{2 g^2}{v_{\infty}} \tanh \left(\frac{g t}{v_{\infty}}\right) \text{sech}^2\left(\frac{g t}{v_{\infty}}\right) \\
s(t) &= \frac{d^3v}{dt^3} = \frac{2 g^3}{v_{\infty}^2} \left(\cosh \left(\frac{2 g t}{v_{\infty}}\right)-2\right) \text{sech}^4\left(\frac{g t}{v_{\infty}}\right) \\
\vdots
\end{align}
$$

In [11]:
accel_ms2 = -9.8 # meter / sec^2
v_inf = 54 # meter / sec
time_steps = 100
delta_time_sec = 0.25  # sec
time_axis_sec = [delta_time_sec * (t-1) for t in 1:time_steps];

In [20]:
plot(
    [
        # Velocity
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[v_inf * tanh(accel_ms2 * t / v_inf) for t in time_axis_sec],
            name="Velocity"
        ),
        # Acceleration
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[accel_ms2 * sech(accel_ms2 * t / v_inf)^2 for t in time_axis_sec],
            name="Acceleration"
        ),
        # Jerk
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[-2 * accel_ms2^2 / v_inf * tanh(accel_ms2 * t / v_inf)^2
               * sech(accel_ms2 * t / v_inf)^2 for t in time_axis_sec],
            name="Jerk"
        ),
        # Snap
        scatter(
            mode="markers+lines",
            x=time_axis_sec,
            y=[2 * accel_ms2^3 / v_inf^2 * (cosh(2 * accel_ms2 * t / v_inf) - 2)
               * sech(accel_ms2 * t / v_inf)^4 for t in time_axis_sec],
            name="Snap"
        ),
    ],
    Layout(
        title="Velocity Model",
        xaxis_title="Time (second)",
        yaxis_title="Arbitrary units",
        legend_title="Legend Title",
    ),
    config=PlotConfig(scrollZoom=true)
)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, title, xaxis, and yaxis"